In [30]:
import json

with open('linkedincreds.json') as f:
    creds = json.load(f)

In [31]:
username = creds['email']
password = creds['password']

In [32]:
def logging(function):
    def wrapper(*args, **kwargs):
        value = function(*args, **kwargs)
        with open('log.txt', 'a+') as file:
            file.write(f'Executed : {function.__name__}, stauts: {value}')
        return value
    return wrapper

In [33]:
# Importing the libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [34]:
# Creating the service
service = Service(r'chromedriver.exe')
chrome_options = Options()
chrome_options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)
# get to linkedin login page
driver.get(r'https://linkedin.com/uas/login')
time.sleep(2)

# passing the creds to the service and logging in
driver.find_element(By.ID, 'username').send_keys(username)
driver.find_element(By.ID, 'password').send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(2)

try:
    captcha = driver.find_element(By.ID, 'captcha-internal')
    if captcha.is_displayed():
        print('Solve the Captcha and press enter')
        input()
except:
    pass

firstname = 'firstname'
lastname = 'lastname'



In [36]:
#Lets get my own profile information
driver.get(r'https://www.linkedin.com/in/ritesh-aie/')
my_page = driver.page_source
with open('my_page.html', 'w+') as f:
    f.write(my_page)


In [7]:
# lets search name now
driver.find_element(By.ID, 'global-nav-search').click()

search_bar_input = driver.find_element(By.XPATH, "//input[@type='text']")
search_bar_input.send_keys(f'{firstname} {lastname}')
search_bar_input.send_keys(Keys.RETURN)


In [8]:
see_all_div = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "search-results__cluster-bottom-banner")))
see_all_div.click()

In [9]:
page_html = driver.page_source


In [11]:
import bs4

soup = bs4.BeautifulSoup(page_html, 'html.parser')

In [ ]:
# with open('parsed.html', 'w+') as f:
#     f.write(page_html)

In [ ]:
# with open('parsed.html', 'r+') as f:
#     html = f.read()

In [ ]:
# soup = bs4.BeautifulSoup(html, 'html.parser')

In [12]:
all_present_persons = soup.find_all('li', class_ = 'reusable-search__result-container')
len(all_present_persons)

10

In [13]:
all_links = []
for person in all_present_persons:
    person_profile_link = person.find('a', class_ = 'app-aware-link')
    # print(person_profile_link)
    all_links.append(person_profile_link['href'])
    # print('\n\n\n\n\n\n\n')

In [14]:
all_links

['https://www.linkedin.com/in/firstname-lastname-2a24141b3?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADG3jZoBh6FdcPa6t-cvNCkx8iZEXE_uP1o',
 'https://www.linkedin.com/in/firstname-lastname-523538222?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADf2x-sBiUkNhxsADPw4eRIWAV68eED6p48',
 'https://www.linkedin.com/in/firstname-lastname-b6131a27?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAWQbvkBQTEHAv9JfFoHMFWwFnNt841V_xE',
 'https://www.linkedin.com/in/firstname-lastname-2a376a313?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAE-rLxsBEtd88vyaMgZkP32TcAbQD2O18OU',
 'https://www.linkedin.com/search/results/people/headless?origin=CLUSTER_EXPANSION&keywords=firstname%20lastname',
 'https://www.linkedin.com/in/firstname-lastname-b65814292?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEb0sn0BqEr-WO6ZNW4CnIToufPz8CKeIgY',
 'https://www.linkedin.com/in/firstname-lastname-60a5412a5?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEmVmioBW1xDxjH9UgC6Gd-RPBJRLxwfA5I',
 'https://www.linkedin

In [15]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
import random

In [20]:
def get_profile_pages(url):
    driver.get(url)
    time.sleep(4)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    _ = wait.until(EC.element_to_be_clickable((By.ID, 'globalfooter-about')))

    page_html = driver.page_source
    _soup = bs4.BeautifulSoup(page_html, 'html.parser')
    name = _soup.find('h1')
    name = name.text
    val = random.randint(0,1999999999)
    with open(f'{name}-{val}.html', 'w+') as f:
        f.write(page_html)
    return f'{name}-{val}.html'

In [22]:
fetched_profile_pages = []
for url in all_links:
    try:
        url_path = get_profile_pages(url)
        fetched_profile_pages.append(url_path)
        print(True)
        print('\n')
    except Exception as e:
        print(False)
        print('\n')
        pass

True


True


True


True


True


True


True


True


True


True




In [ ]:
def get_summary(soup):
    try:
        summary = soup.find('h2', class_ = 'top-card-layout__headline break-words font-sans leading-open text-color-text')
    except:
        try:
            summary = soup.find('div', 'text-body-medium break-words')
        except:
            summary = None
    return summary

def get_education(soup):
    education_list = []
    
    try:
        education_container = soup.select_one('#education + div + div')
        
        education_container_list = education_container.find_all('li')
        
        for education in education_container_list:
            education_dict = {}

            try:
                education_center = education.find('div', class_='justify-space-between')
                
                try:
                    education_link = education_center.find('a', href=True)['href']
                except:
                    education_link = None
                
                try:
                    education_institute = education_center.find('div', class_='display-flex').text.strip()
                except:
                    education_institute = None
                
                try:
                    education_detail_container = education_center.find_all('div', class_='t-14 t-normal')
                    education_branch = education_detail_container[0].text.strip() if len(education_detail_container) > 0 else None
                    education_timeline = education_detail_container[1].text.strip() if len(education_detail_container) > 1 else None
                except:
                    education_branch = None
                    education_timeline = None
            except:
                print('No Container Found')
                education_link = None
                education_institute = None
                education_branch = None
                education_timeline = None
            
            education_dict['link'] = education_link
            education_dict['institute'] = education_institute
            education_dict['domain'] = education_branch
            education_dict['timeline'] = education_timeline
            education_list.append(education_dict)
    except Exception as e:
        print(f"An error occurred: {e}")
        education_list = []
    
    education_list_str = [str(edu) for edu in education_list]
    return education_list_str


def get_experience(soup):
    experience_list = []
    try:
        experience_div = soup.find()


In [ ]:
def get_details(html_path):
    'full_name, details, connection_level, activity_links'
    with open(html_path, 'r') as f:
        html_data = f.read()
        soup = bs4.BeautifulSoup(html_data, 'lxml')
        name_detail_container = soup.find('div', class_='ph5')
        full_name = name_detail_container.find('h1')
        details = name_detail_container.find('div', class_='text-body-medium')
        connection_level = name_detail_container.find('span', class_ = 'link-without-visited-state')
        all_activities = soup.find_all('li', class_ = 'profile-creator-shared-feed-update__mini-container')
        activity_links = []
        for activity in all_activities:
            activity_a = activity.find('a', href=True)
            link = activity_a['href']
            activity_links.append(link)

    
